## AutoEncoder

*Project Description*
- Develop a model to predict a users next movie rating.


Importing Necessary Libraries

In [21]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Set up device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### 1) Data Handling

Below dataset is used to train autoencoder on 1 million user ratings.
The current model is tested off a dataset containing 100k user ratings.

In [22]:
# We won't be using this dataset. 
# movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

Creating the Testing and Training sets.

In [23]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header = None)
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header = None)
test_set = np.array(test_set, dtype = 'int')

In [24]:
#%% Getting the number of users and movies

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

mins = int(min(min(training_set[:, 1], ), min(test_set[:, 1])))

In [25]:
#%% Converting the data into an array with users in lines and movies in columns

def convertData(data:np.array):
    """ 
        Convert training data into 2D matrix where row indicies equal users and
        columns equal movie id. Cell reffers to user rating.
        
        Fill 0 for user who did not rate a movie.
        
        Information from training data:
        Column 1: User -> min user = 1, max user = 943
        Column 2: Movie -> min id = 1, max id = 1682
        Column 3: Rating
        Column 4: Timestamp
    
    """
    convert_data = []
    for user in range(1, nb_users + 1):
        zeros = [0] * (nb_movies + 1)
        single_user_reviews = data[data[:, 0] == user]
        for rating in single_user_reviews:
            zeros[rating[1]] = rating[2]
        zeros.pop(0) #no movie id that = 0
        convert_data.append(zeros)
    
    return convert_data

training_set = convertData(training_set)
test_set = convertData(test_set)

In [72]:
#%% Converting the data into Torch tensors

training_set = torch.tensor(training_set, dtype=torch.float).to(device)
test_set = torch.tensor(test_set, dtype=torch.float).to(device)

C:\Users\btehrani\AppData\Local\Temp\ipykernel_55164\2225292254.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_set = torch.tensor(training_set, dtype=torch.float).to(device)
C:\Users\btehrani\AppData\Local\Temp\ipykernel_55164\2225292254.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_set = torch.tensor(test_set, dtype=torch.float).to(device)


### 2) Develop Model

In [73]:
#%% Creating the architecture of the Neural Network

class SAE(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(nb_movies, 20),
            nn.Linear(20, 10),
            nn.Linear(10, 20),
            nn.Linear(20, nb_movies)
        )
        #self.activation = nn.Sigmoid()
    
    def forward(self, x):   
       return self.layers(x)
   
sae = SAE().to(device)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay= 0.5)   

print(f"Model is on device: {device}")

Model is on device: cuda


### 3) Train Model

In [88]:
nb_epoch = 200

for epoch in range(1, nb_epoch + 1):
    
    train_loss = 0

    # Count number of users that rated 1 movie
    # Do not count users who did not rate any movie for memory optimization
    s = 0. # Will use to compute RMS error

    for id_user in range(nb_users):
        input_x = Variable(training_set[id_user]).unsqueeze(0)
        target = input_x.clone()

        if torch.sum(target.data > 0) > 0:
            
            output = torch.sigmoid(sae(input_x))
            #output = sae(input_x)

            # Optimize code to not compute gradients on target, only input tensor
            target.require_grad = False
           
            """ 
                Below is necessary for optimization and model performance.

                If a user did not rate a movie and the model predicts their rating to be high,
                the model will calulate this RMS error and will adjust inappropriately:

                ex/ User did not rate movie = 0 
                    Model predicts = 3

                    This result will carry over to loss and assume a loss of 3. 
                    Model loss will theerefore have a difficult time decreasing.
            """
            # output[target == 0] = 0
            output_2 = torch.where(target == 0, torch.tensor(0.), output)

            # output_2 = output.clone()
            # output_2[target == 0] = 0

            loss = criterion(output_2, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            
            loss.backward()
            train_loss = train_loss + np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step()

    #if epoch % 10 == 0:
    print('epoch: ' + str(epoch) + ' loss: '+ str(train_loss/s))

epoch: 1 loss: 2.932022894989428
epoch: 2 loss: 2.8846614639011885
epoch: 3 loss: 2.874855224822831
epoch: 4 loss: 2.8689575678661106
epoch: 5 loss: 2.86552831826725
epoch: 6 loss: 2.8615621762483507
epoch: 7 loss: 2.859476122865934
epoch: 8 loss: 2.857108151439103
epoch: 9 loss: 2.8552896999774857
epoch: 10 loss: 2.8538914339525774
epoch: 11 loss: 2.8520950116738675
epoch: 12 loss: 2.850581408586422
epoch: 13 loss: 2.8491413325258934
epoch: 14 loss: 2.8477188911910347
epoch: 15 loss: 2.846965920623003
epoch: 16 loss: 2.845937310928483
epoch: 17 loss: 2.8450520465930422
epoch: 18 loss: 2.8430517674175775
epoch: 19 loss: 2.8411078441514634
epoch: 20 loss: 2.8396996253688926
epoch: 21 loss: 2.8377880306557453
epoch: 22 loss: 2.8357254228698006
epoch: 23 loss: 2.8341613461699446
epoch: 24 loss: 2.8308008912570686
epoch: 25 loss: 2.8112551747333767
epoch: 26 loss: 2.795532916959039
epoch: 27 loss: 2.7951498645220285
epoch: 28 loss: 2.794929168894925
epoch: 29 loss: 2.7949151606574487
epoch

KeyboardInterrupt: 

### 4) Test Model

In [ ]:
#%% Testing the SAE

test_loss = 0
s = 0.
for id_user in range(nb_users):
    input_x = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input_x)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.

print('test loss: '+ str(test_loss/s))

test loss: 0.9482630600945305



Model takes a while to train on most systems.
Save / Load model state after training in case errors occur.

In [ ]:
torch.save(sae, '/model_sae.pt')